we had 140000 images approximately for our final dataset based on https://arxiv.org/pdf/1812.10595.pdf. We'll try to make it smaller.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("/storage"))

# Any results you write to the current directory are saved as output.


import os, sys
import re
from multiprocessing import Pool
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage.io
from skimage.transform import resize
from scipy import ndimage
from imgaug import augmenters as iaa
from tqdm import tqdm
import PIL
from PIL import Image, ImageOps
import cv2
from sklearn.utils import class_weight, shuffle
from sklearn.metrics import f1_score, fbeta_score
from sklearn.model_selection import train_test_split
import seaborn as sns
from collections import Counter
import imageio

from torchvision.transforms import transforms

WORKERS = 2
CHANNEL = 3

import warnings
warnings.filterwarnings("ignore")
IMG_SIZE = 512
NUM_CLASSES = 5
SEED = 77
TRAIN_NUM = 1000 # use 1000 when you just want to explore new idea, use -1 for full train

['aptosplus.zip', 'trainLabels_aug.csv', 'retinopathy-train-2015', '.ipynb_checkpoints', 'lost+found', 'aptosplus', 'diabetic-retinopathy-resized', 'aptos2019-blindness-detection', 'resnet101.pth']


In [26]:
df_train_aug = pd.read_csv('/storage/aptosplus/trainLabels_aug.csv')
df_train = pd.read_csv('/storage/aptosplus/trainLabels.csv')

In [3]:
df_aptos = df_train.loc[df_train['ds'] == 'aptos']

In [4]:
df_aptos.tail()

,id_code,diagnosis,ds
3657,ffa47f6a7bf4,2,aptos
3658,ffc04fed30e6,0,aptos
3659,ffcf7b45f213,2,aptos
3660,ffd97f8cd5aa,0,aptos
3661,ffec9a18a3ce,2,aptos


In [5]:
df_eyepacs = df_train.loc[df_train['ds'] == 'eyepacs']

In [6]:
df_eyepacs.tail()

,id_code,diagnosis,ds
38783,44347_right,0,eyepacs
38784,44348_left,0,eyepacs
38785,44348_right,0,eyepacs
38786,44349_left,0,eyepacs
38787,44349_right,1,eyepacs


In [7]:
df_train.groupby('diagnosis').agg(['count'])

,id_code,ds
,count,count
diagnosis,,
0,27615,27615
1,2813,2813
2,6291,6291
3,1066,1066
4,1003,1003


In [8]:
df_aptos.groupby('diagnosis').agg(['count'])

,id_code,ds
,count,count
diagnosis,,
0,1805,1805
1,370,370
2,999,999
3,193,193
4,295,295


In [9]:
df_eyepacs.groupby('diagnosis').agg(['count'])

,id_code,ds
,count,count
diagnosis,,
0,25810,25810
1,2443,2443
2,5292,5292
3,873,873
4,708,708


We want:

- All the images from aptos
- In eyepacs:
    - 3200 images for 0 (done)
    - all images for a total of 2813 images
    - 4000 images (done)
    - all images for a total of 1000 images
    - all images for a total of 1000 images

In [13]:
df_final = pd.concat([
    df_aptos,
    df_eyepacs.loc[df_eyepacs['diagnosis'] == 0].sample(n=3200, random_state=SEED),
    df_eyepacs.loc[df_eyepacs['diagnosis'] == 1],
    df_eyepacs.loc[df_eyepacs['diagnosis'] == 2].sample(n=4000, random_state=SEED),
    df_eyepacs.loc[df_eyepacs['diagnosis'] == 3],
    df_eyepacs.loc[df_eyepacs['diagnosis'] == 4]
    ])

In [14]:
print(len(df_final))

14886


In the augmented images, we need to take:

- 2200 images for 1
- 4000 images for 3
- 4000 images for 4

In [27]:
df_train_aug = df_train_aug[df_train_aug['id_code'].str.contains('_a')]

In [29]:
df_final = pd.concat([
    df_final,
    df_train_aug.loc[df_train_aug['diagnosis'] == 1].sample(n=2200, random_state=SEED),
    df_train_aug.loc[df_train_aug['diagnosis'] == 3].sample(n=4000, random_state=SEED),
    df_train_aug.loc[df_train_aug['diagnosis'] == 4].sample(n=4000, random_state=SEED),
])

In [30]:
print(len(df_final))

25086


In [35]:
df_valid = pd.concat([
    df_final.loc[df_final['diagnosis'] == 0].sample(n=100, random_state=SEED),
    df_final.loc[df_final['diagnosis'] == 1].sample(n=100, random_state=SEED),
    df_final.loc[df_final['diagnosis'] == 2].sample(n=100, random_state=SEED),
    df_final.loc[df_final['diagnosis'] == 3].sample(n=100, random_state=SEED),
    df_final.loc[df_final['diagnosis'] == 4].sample(n=100, random_state=SEED),
])

In [42]:
df_final_wo_valid = df_final[~df_final.isin(df_valid)].dropna()

In [44]:
df_valid.to_csv('/storage/aptosplus/validLabels_smallds.csv')

In [45]:
df_final_wo_valid.to_csv('/storage/aptosplus/trainLabels_smallds.csv')